In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import re
import string
import nltk
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


ModuleNotFoundError: No module named 'nltk'

In [260]:
train_percentage = 0.3

###### Custom Metric Cosine

In [261]:
def cosine_dist(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return 1 - cos_sim

### LOAD DATA

In [262]:
FILENAME = "data/database_kara.csv"          

dataset = pd.read_csv(FILENAME, header = None)

dataset.rename(columns = {0:'text', 1:'label'}, inplace = True)

dataset.dropna(how='any', inplace=True)
dataset.reset_index(drop=True, inplace=True)

dataset['en_label'] = dataset["label"]
for i in range(dataset.shape[0]):
    label = dataset.loc[i,"en_label"]
    label = re.sub("[Yy]", "", label)
    # delete rows with wrong type of label
    if type(label) == float or label == "" or label == " ":
        dataset.drop([i], axis=0, inplace=True)
        continue
    dataset.loc[i,"en_label"] = label
    
# Shape = (sentence_count, 3) : sentence, label, encoded label

In [263]:
dataset

,text,label,en_label
0,Birçok toplum ilham ve sezişlerine rüyalarına ...,Y2,2
1,Günümüzde Aristo’nun bilimsel bilgisinden fazl...,Y2,2
2,Bilimin ilerlemesi bilinmeyenler diyarına deva...,Y2,2
3,İçinde bulunduğumuz kara parçası dünyada birço...,Y1,1
4,Kara duvara aşkını kazırken düşündüğü şey atmo...,Y2,2
...,...,...,...
1514,Şimdi birer düş gibi anımsıyordu bütün bunları...,Y2,2
1515,Ne kendi kendine ne de akasyaların kara gölges...,Y2,2
1516,Her zamanki yolundan değil otlaktan geçerek ay...,Y2,2
1517,Upuzun kara saçları esmer teni dans ettikçe pı...,Y2,2


In [264]:
del dataset["label"]

In [265]:
dataset.reset_index(drop=True, inplace=True)

In [266]:
dataset.isnull().values.any()

False

In [267]:
dataset

,text,en_label
0,Birçok toplum ilham ve sezişlerine rüyalarına ...,2
1,Günümüzde Aristo’nun bilimsel bilgisinden fazl...,2
2,Bilimin ilerlemesi bilinmeyenler diyarına deva...,2
3,İçinde bulunduğumuz kara parçası dünyada birço...,1
4,Kara duvara aşkını kazırken düşündüğü şey atmo...,2
...,...,...
1513,Şimdi birer düş gibi anımsıyordu bütün bunları...,2
1514,Ne kendi kendine ne de akasyaların kara gölges...,2
1515,Her zamanki yolundan değil otlaktan geçerek ay...,2
1516,Upuzun kara saçları esmer teni dans ettikçe pı...,2


### PREPROCESSING

In [268]:
s = set(stopwords.words('turkish'))
ps = nltk.wordnet.WordNetLemmatizer()

for i in range(dataset.shape[0]):
    review = dataset.loc[i,'text']
    for punc in string.punctuation:
        review.replace(punc, "")
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in s]
    #review = [stemmer.stem(word) for word in review if not word in s]
    review = ' '.join(review)
    dataset.loc[i, 'text'] = review

In [269]:
dataset.dropna(how='any', inplace=True)
dataset.reset_index(drop=True, inplace=True)

In [270]:
dataset

,text,en_label
0,birçok toplum ilham sezişlerine rüyalarına gai...,2
1,günümüzde aristo’nun bilimsel bilgisinden fazl...,2
2,bilimin ilerlemesi bilinmeyenler diyarına deva...,2
3,i̇çinde bulunduğumuz kara parçası dünyada birç...,1
4,kara duvara aşkını kazırken düşündüğü atmosfer...,2
...,...,...
1513,şimdi birer düş anımsıyordu bütün bunları çarp...,2
1514,kendi kendine akasyaların kara gölgesinde dişl...,2
1515,zamanki yolundan değil otlaktan geçerek ay ışı...,2
1516,upuzun kara saçları esmer teni dans ettikçe pı...,2


###### VECTORIZE TEXT

In [271]:
count_vect = CountVectorizer()
count_vectors = count_vect.fit_transform(dataset["text"])

In [272]:
count_vectors

<1518x8525 sparse matrix of type '<class 'numpy.int64'>'
	with 18236 stored elements in Compressed Sparse Row format>

##### SPLIT DATA

In [273]:
X_train, X_test, y_train, y_test = train_test_split(count_vectors, dataset["en_label"], test_size=(1-train_percentage))

### MODEL TRAINING

In [274]:
model = KNeighborsClassifier(n_neighbors=30, metric="euclidean")
model.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=30)

In [275]:
y_predict = model.predict(X_test)

In [276]:
y_predict = np.asarray(y_predict, dtype=int)
y_test = np.asarray(y_test, dtype=int)

In [277]:
y_predict

array([2, 2, 2, ..., 2, 2, 2])

In [278]:
accuracy_score(y_test,y_predict)

0.7808090310442145

In [279]:
f1_score(y_test, y_predict, average="weighted")

0.6847031122733207